In [10]:
# Тюнинг
import optuna as opt
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans

# Пайплайн
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

# Данные
import os
import pandas as pd
import numpy as np
from category_encoders import BinaryEncoder
from sklearn.preprocessing import RobustScaler

In [11]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
BALANCED_DATASET = os.path.join(ROOT, '../data/balanced_train.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')

# Загрузка
train_df = pd.read_csv(TRAIN_DATASET)
balanced_df = pd.read_csv(BALANCED_DATASET, index_col=0)
test_df = pd.read_csv(TEST_DATASET)

X, y = balanced_df.iloc[:, :-1], balanced_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

features_to_drop = [
    'Изменение позиции заказа на закупку: изменение даты поставки на бумаге',
    'Количество', 'Категорийный менеджер'
    ]

X_train = X_train.drop(features_to_drop, axis=1)
X_test = X_test.drop(features_to_drop, axis=1)
test_df = test_df.drop(features_to_drop, axis=1)

In [12]:
cat_features = [
    'Поставщик', 'Материал', 'Операционный менеджер',
    'Завод', 'Закупочная организация', 'Группа закупок', 'Балансовая единица',
    'ЕИ', 'Вариант поставки'
    ]

In [13]:
# Препроцессоры
class DataPreprocessor(BaseEstimator, TransformerMixin):
    """ Предобработчик данных """
    def __init__(self, cat_features, transform_train=True):
        self.transform_train = transform_train
        self.bin_encoder = BinaryEncoder(cols=cat_features)
        self.robust_scaler = RobustScaler()

    def fit(self, X, y=None):
        # Создаём копию датасета
        X_ = X.copy()

        # Временные фичи
        # X_['day_sin'] = np.sin(np.pi * 2 * X_['День недели 2'] / 6)
        # X_['day_cos'] = np.cos(np.pi * 2 * X_['День недели 2'] / 6)

        # Категориальные фичи
        X_ = self.bin_encoder.fit_transform(X_)

        # Нумерация фич для LGBM (не принимает JSON символы)
        X_.columns = [num for num in range(0, len(X_.columns))]

        # Масштабирование
        self.robust_scaler.fit(X_)

        return self
    
    def transform(self, X):
        # Создаём копию датасета
        X_ = X.copy()

        # Временные фичи
        # X_['day_sin'] = np.sin(np.pi * 2 * X_['День недели 2'] / 6)
        # X_['day_cos'] = np.cos(np.pi * 2 * X_['День недели 2'] / 6)

        # Категориальные фичи
        X_ = self.bin_encoder.transform(X_)

        # Нумерация фич для LGBM (не принимает JSON символы)
        X_.columns = [num for num in range(0, len(X_.columns))]

        # Масштабирование
        X_ = self.robust_scaler.transform(X_)

        return X_


In [19]:
# data_preprocessor.fit(X_train)
# X_train_preproc = data_preprocessor.transform(X_train)
# X_test_preproc = data_preprocessor.transform(X_test)

In [6]:
# Функция оптимизации
def objective(trial: opt.Trial):
    # Параметры
    learning_rate = trial.suggest_float('learning_rate', 0.1, 1, log=True)
    n_estimators = trial.suggest_int('n_estimators', 1000, 3000, 50)
    max_depth = trial.suggest_int('max_depth', 6, 24)
    max_bin = trial.suggest_int('max_bin', 64, 256),
    num_leaves = trial.suggest_int('num_leaves', 64, 512)
    reg_lambda = trial.suggest_float('l2_reg', 0.1, 1.5, log=True)

    # Модель
    data_preprocessor = DataPreprocessor(cat_features)
    model = LGBMClassifier(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        reg_lambda=reg_lambda,
        max_bin=max_bin,
        n_jobs=-1,
        force_col_wise=True,
        verbose=-1
    )

    pipeline = Pipeline([
        ('data_preproc', data_preprocessor),
        ('model', model)
    ])

    pipeline.fit(X_train, y_train)
    accuracy = f1_score(y_test, pipeline.predict(X_test), average='macro')
    
    # cv_score = cross_val_score(model, X_train_lgbm, y_train, cv=StratifiedKFold(), scoring='f1_macro', n_jobs=-1)
    # accuracy = cv_score.mean()

    return accuracy

In [ ]:
study = opt.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

In [16]:
best_params = {
    'learning_rate': 0.2945309998327077, 
    'n_estimators': 1500, 
    'max_depth': 21, 
    'max_bin': 204, 
    'num_leaves': 304, 
    'reg_lambda': 0.762011415827371
    }

# Модель
data_preprocessor = DataPreprocessor(cat_features)
model = LGBMClassifier(
    **best_params,
    n_jobs=-1,
    force_col_wise=True,
)

pipeline = Pipeline([
    ('data_preproc', data_preprocessor),
    ('model', model)
])

In [17]:
pipeline.fit(X_train, y_train)
f1_score(y_test, pipeline.predict(X_test))

[LightGBM] [Info] Number of positive: 116489, number of negative: 173320
[LightGBM] [Info] Total Bins 2651
[LightGBM] [Info] Number of data points in the train set: 289809, number of used features: 97
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.401951 -> initscore=-0.397343
[LightGBM] [Info] Start training from score -0.397343
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


0.9841196954042781

In [18]:
f1_score(y_test, pipeline.predict(X_test), average='macro')

0.9866379878082245

In [19]:
preds = pipeline.predict(test_df)
submit_df = pd.DataFrame({'id': test_df.index, 'value': preds})

In [20]:
submit_df.to_csv('submission_6.csv', index=False)

In [29]:
submit_df

,id,value
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
24995,24995,0
24996,24996,1
24997,24997,1
24998,24998,1
